<a href="https://colab.research.google.com/github/Jalam1001/AI_AGENT_RESEARCH/blob/main/crewai_gemini_stock_analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install crewai

  Using cached langchain_core-0.1.52-py3-none-any.whl (302 kB)
  Attempting uninstall: langchain-core
    Found existing installation: langchain-core 0.2.1
    Uninstalling langchain-core-0.2.1:
      Successfully uninstalled langchain-core-0.2.1
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
langchain-google-genai 1.0.5 requires langchain-core<0.3,>=0.2.0, but you have langchain-core 0.1.52 which is incompatible.


In [2]:
!pip install langchain_google_genai

  Using cached langchain_core-0.2.1-py3-none-any.whl (308 kB)
  Attempting uninstall: langchain-core
    Found existing installation: langchain-core 0.1.52
    Uninstalling langchain-core-0.1.52:
      Successfully uninstalled langchain-core-0.1.52
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
langchain 0.1.20 requires langchain-core<0.2.0,>=0.1.52, but you have langchain-core 0.2.1 which is incompatible.
langchain-community 0.0.38 requires langchain-core<0.2.0,>=0.1.52, but you have langchain-core 0.2.1 which is incompatible.


In [3]:
import os
from langchain_google_genai import ChatGoogleGenerativeAI
from crewai import Agent, Task, Crew, Process

In [4]:
from google.colab import userdata
# Set the environment variable for the Google API key
os.environ["GOOGLE_API_KEY"] = userdata.get('GOOGLE_API_KEY')


# Set gemini pro as llm
llm = ChatGoogleGenerativeAI(model="gemini-pro", verbose=True, temperature=0.6)

In [5]:
researcher = Agent(
    role="Senior Finance Analyst",
    goal="Fetch the stock value of Tesla Inc.",
    backstory="""You work at a leading investment banking think tank.
  Your expertise lies analysis data and you have a knack for dissecting complex data and presenting
  actionable insights. Keep the report simple and easy to understand within 300 words""",
    verbose=True,
    allow_delegation=False,
    llm=llm,
)

In [6]:


analyst = Agent(
    role="Investor",
    goal="Highlight the key metrics of Tesla Inc. stock value",
    backstory="""You are a renowned analyst and investor, known for
  your number crunching and insights from data. You transform complex concepts into compelling narratives.""",
    verbose=True,
    allow_delegation=False,
    llm=llm,
)

In [7]:
news = Agent(
    role="news collector",
    goal="collect all the relevant news on tesla",
    backstory="""You are a news journalist, known for your financial news gathering and analysis
  your news gathering and insights from news. You transform complex concepts into compelling narratives.""",
    verbose=True,
    allow_delegation=False,
    llm=llm,
)

In [8]:
designer = Agent(
    role="Visualizer",
    goal="Visualize the key metrics from task2 using a graph",
    backstory="""You are a renowned graphics designer known for amazing visualization of numbers and great story telling""",
    verbose=True,
    allow_delegation=False,
    llm=llm,
)

In [9]:
# Create tasks for your agents
task1 = Task(
    description="""Conduct a comprehensive analysis, get the key stock metrics and
  your final answer MUST be a paragraph not exceeding 100 words.""",
    expected_output="""A detailed report summarizing key findings highlighting information that could be relevant for the research.""",
    agent=researcher,
)

In [10]:
task2 = Task(
    description="""Using the content from task1, highlight key metrics.
  Avoid complex words so it doesn't sound like AI.
  """,
    expected_output="""A blog highlighting findings from the research.""",
    agent=analyst,
)

In [11]:
task3 = Task(
    description="""get the latest news on tesla""",
    expected_output="""write a 1000 word report using the findings from the news.""",
    agent=designer,
)


In [12]:
task4 = Task(
    description="""Visualize the key metrics from task2 using a graph""",
    expected_output="""write a python code and create a chart using the findings from the research.""",
    agent=designer,
)

In [13]:
# Instantiate your crew with a sequential process
crew = Crew(
    agents=[researcher, analyst, news, designer],
    tasks=[task1, task2, task3, task4],
    verbose=1,
)

In [14]:
# Get your crew to work!
result = crew.kickoff()

 [DEBUG]: == Working Agent: Senior Finance Analyst


> Entering new CrewAgentExecutor chain...
Thought: I now can give a great answer
Final Answer: Tesla, Inc. (NASDAQ: TSLA) is an American electric vehicle and clean energy company based in Palo Alto, California. The company was founded in 2003 by Martin Eberhard and Marc Tarpenning as Tesla Motors. Tesla designs, develops, manufactures, and sells electric cars, solar panels, and batteries. The company also provides energy storage solutions and other related products and services.

As of March 8, 2023, Tesla's stock price is $207.54 per share. The company's market capitalization is $658.86 billion. Tesla's stock has been on a roller coaster ride in recent years, but it has generally trended upwards. The company's stock price has increased by over 1,000% in the past five years.

Tesla is a controversial company, but it is also one of the most successful companies in the world. The company's mission is to accelerate the world's transitio

In [16]:
from IPython.display import Markdown
Markdown(result)

```
import matplotlib.pyplot as plt

# Create the data
x = ['Stock Price', 'Market Capitalization', 'Revenue', 'Net Income', 'Gross Margin', 'Operating Margin', 'Net Margin', 'Return on Equity', 'Debt-to-Equity Ratio']
y = [200, 658.86, 81.46, 12.6, 29.2, 16.8, 15.5, 22.5, 0.5]

# Create the bar chart
plt.bar(x, y)

# Add a title and labels
plt.title('Tesla Key Metrics')
plt.xlabel('Metric')
plt.ylabel('Value')

# Show the chart
plt.show()
```